In [11]:
# -*- coding: utf-8 -*-
# VGG模型

import torchvision
from torch.utils.tensorboard import SummaryWriter

# 导入VGG模型
from VGGmodel import *

# 准备数据集
from torch import nn
from torch.utils.data import DataLoader

# from VGG

train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)

# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
# 如果train_data_size=10, 训练数据集的长度为：10
print(f"训练数据集的长度为：{train_data_size}")
print(f"测试数据集的长度为：{test_data_size}")

# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型
vgg = VGG()

# 损失函数
loss_fn = nn.CrossEntropyLoss()

# 创建网络模型


class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        # 网络序列
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


vgg = VGG()

# 损失函数
loss_fn = nn.CrossEntropyLoss()

# 优化器
# 调整速率
learning_rate = 0.01
# 1e-2=1 x (10)^(-2) = 1 /100 = 0.01
learning_rate = 1e-2
optimizer = torch.optim.SGD(vgg.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 3

# 添加tensorboard
writer = SummaryWriter("./logs_VGGtrain")

for i in range(epoch):
    print(f"-------第 {i+1} 轮训练开始-------")

    # 训练步骤开始
    vgg.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = vgg(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        # 每一百次输出
        if total_train_step % 100 == 0:
            # item()转化为数字
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    vgg.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            outputs = vgg(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy /
                      test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(
        vgg, "/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/Pytorch/demo1/vggmodel/vgg_{}.pth".format(i))
    print("模型已保存")

writer.close()


Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-------第 1 轮训练开始-------
训练次数：100, Loss: 2.2849576473236084
训练次数：200, Loss: 2.2824320793151855
训练次数：300, Loss: 2.2445690631866455
训练次数：400, Loss: 2.1626944541931152
训练次数：500, Loss: 2.079289674758911
训练次数：600, Loss: 2.07782244682312
训练次数：700, Loss: 2.0028035640716553
整体测试集上的Loss: 313.51774632930756
整体测试集上的正确率: 0.2847000062465668
模型已保存
-------第 2 轮训练开始-------
训练次数：800, Loss: 1.879063367843628
训练次数：900, Loss: 1.8667560815811157
训练次数：1000, Loss: 1.9377219676971436
训练次数：1100, Loss: 1.9656537771224976
训练次数：1200, Loss: 1.7167010307312012
训练次数：1300, Loss: 1.655203938484192
训练次数：1400, Loss: 1.7676793336868286
训练次数：1500, Loss: 1.832297921180725
整体测试集上的Loss: 316.5826255083084
整体测试集上的正确率: 0.2912999987602234
模型已保存
-------第 3 轮训练开始-------
训练次数：1600, Loss: 1.7543716430664062
训练次数：1700, Loss: 1.6703763008117676
训练次数：1800, Loss: 1.9497393369674683
训练次数：1900, Loss: 1.6920299530029297
训练次数：2000, Los

In [1]:
!nvidia-smi

Mon Mar 14 20:42:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 48%   42C    P2    53W / 165W |   1266MiB /  6144MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------